In [2]:
%load_ext sql
%sql sqlite:///

/Library/Python/2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Library/Python/2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@'

Problem Set 2
=======


### Instructions / Notes:

**_Read these carefully_**

* This problem set _does not_ come with a dataset to load; instead, make your own instances of tables, either as solutions to the problems or for testing solutions to the problems.
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* **See Piazza for submission instructions**
* _Have fun!_

Problem 1
---------

**_[20 points total]_**

For each part of this problem you will need to provide a _single_ SQL query which will check whether a certain condition holds on a specific instance of a relation, in the following way: **your query should return an empty result if and only if the condition holds on the instance.**  (If the condition _doesn't hold_, your query should return something non-empty, but it doesn't matter what this is).

Note our language here: the conditions that we specify cannot be proved to hold **in general** without knowing the externally-defined functional dependencies; so what we mean is, _check whether they **could** hold in general for the relation, given any specific set of tuples_.

You may assume that there will be no `NULL` values in the tables, **and you may assume that the relations are _sets_ rather than multisets**, but otherwise your query should work for general instances.  We define the schemas of the tables used below for convenience, but in this problem you will need to construct your own test tables if you wish to use them to check your answers!

In [6]:
%%sql
DROP TABLE IF EXISTS T; CREATE TABLE T (A INT, B INT, C INT, D INT);
DROP TABLE IF EXISTS Dog; CREATE TABLE Dog(dog_name TEXT, breed TEXT, owner_name TEXT);
DROP TABLE IF EXISTS Owner; CREATE TABLE Owner(owner_name TEXT, ssn INT, hometown TEXT);
DROP TABLE IF EXISTS S; CREATE TABLE S (A INT, B INT, C INT);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

### Part (a)

**_[5 points]_**

$\{A, B\}$ is a **superkey** for a relation $T(A,B,C,D)$.

In [18]:
%%sql
SELECT *
FROM T AS T1, T AS T2
WHERE T1.A = T2.A AND T1.B = T2.B
    AND (T1.C <> T2.C OR T1.D <> T2.D);

Done.


A,B,C,D,A_1,B_1,C_1,D_1


### Part (b)

**_[5 points]_**

The individual attributes of a relation $T(A,B,C,D)$ are each keys.

In [19]:
%%sql
SELECT *
FROM T AS T1, T AS T2
WHERE (T1.A = T2.A OR T1.B = T2.B OR T1.C = T2.C OR T1.D = T2.D)
    AND (T1.A <> T2.A OR T1.B <> T2.B OR T1.C <> T2.C OR T1.D <> T2.D);

Done.


A,B,C,D,A_1,B_1,C_1,D_1


### Part (c)

**_[5 points]_**

A **multivalued dependency (MVD)** is defined as follows: let $R$ be a schema i.e. a set of attributes, and consider two _sets of attributes_ $X\subseteq R$ and $Y\subseteq R$.  We say that a multivalued dependency (MVD), written:

$X\twoheadrightarrow Y$

**holds on $R$** if whenever there are two tuples $t_1,t_2$ such that $t_1[X] = t_2[X]$, there also exists a third tuple $t_3$ such that:

* $t_3[X] = t_1[X] = t_2[X]$
* $t_3[Y] = t_1[Y]$
* $t_3[R\setminus Y] = t_2[R\setminus Y]$

Note that $R\setminus Y$ is all the attributes in $R$ that are not in $Y$, and that $t_3$ need not be distinct from $t_1$ or $t_2$.  Note especially that an MVD holds on an entire _relation_, meaning that any two tuples (in any order) in the relation should satisfy the above conditions if the MVD holds.  **See the end of the lecture 7 slides for more on MVDs!**


In this problem, write your query to check if the MVD $\{A\}\twoheadrightarrow \{B,D\}$ holds for a relation $T(A,B,C,D)$

In [21]:
%%sql
SELECT *
FROM T AS T1, T AS T2
WHERE T1.A = T2.A
    AND NOT EXISTS (
    SELECT *
    FROM T AS T3
    WHERE T1.A = T3.A AND T1.B = T3.B AND T1.D = T3.D
        AND T2.C = T3.C
    );

Done.


A,B,C,D,A_1,B_1,C_1,D_1


### Part (d)

**_[5 points]_**

A _tuple-generating dependency (TGD)_ between two relations $A$ and $B$, having some shared attributes $X_1,...,X_n$, holds if, for every tuple $t_A$ in $A$, there is _some_ tuple $t_B$ in $B$ such that $t_A[X_i] = t_B[X_i]$ for $i=1,...n$.

In other words, for every distinct tuple in $A$, there must exist a corresponding tuple in $B$, which has the same values of shared attributes.

Consider two tables `Dog(dog_name, breed, owner_name)` and `Owner(owner_name, ssn, hometown)`; check for a TGD between them:

In [7]:
%%sql
SELECT * FROM Dog AS d
WHERE NOT EXISTS (
    SELECT * FROM Owner AS p
    WHERE d.owner_name = p.owner_name);

Done.


dog_name,breed,owner_name


Problem 2
---------

**_[20 points total]_**

### Part (a)

**_[10 points]_**

Consider a relation $R(A,B,C,D,E)$.  Provide _two different sets_ of functional dependencies, `F_1` and `F_2`, such that each one results in $R$ having the **largest number of distinct keys** $R$ could possibly have.

Store your lists of FDs as python lists having elements that are _pairs of sets_; for example to set `F_1` as the set consisting of two FDs, $\{A,B\}\rightarrow\{C,D\}$ and $\{B\}\rightarrow\{C\}$:

```python
F_1 = [(set(['A','B']), set(['C','D'])), (set(['B']), set(['C']))]
```

*Note: the above is not necessarily one of the FDs- just an example of the syntax!

*Hint: You may use any of the functions from the lecture activities if they seem helpful!  However your final answer should not involve these functions directly, nor are they necessary for this problem

*Hint: See Activity 5-3...

In [1]:
F_1 = [
    (set(['A', 'B']), set(['C','D','E'])),
    (set(['A', 'C']), set(['B','D','E'])),
    (set(['A', 'D']), set(['C','B','E'])),
    (set(['A', 'E']), set(['C','D','B'])),
    (set(['B', 'C']), set(['A','D','E'])),
    (set(['B', 'D']), set(['A','C','E'])),
    (set(['B', 'E']), set(['A','D','C'])),
    (set(['C', 'D']), set(['A','B','E'])),
    (set(['C', 'E']), set(['A','D','B'])),
    (set(['D', 'E']), set(['A','B','C'])),
]

F_2 = [(rhs,lhs) for lhs,rhs in F_1]

### Part (b)

**_[10 points]_**

Consider a schema $R(A_1,...,A_n)$ which has FDs $\{A_i,A_{i+1}\}\rightarrow\{A_{i+2}\}$ for $i=1,...,n-2$.  Create an instance of $R$, for $n=4$, for which these FDs hold, and no other ones do- i.e. **all other FDs are violated.**

Use a series of `INSERT` statements below to populate the table `R`:

*Solution overview:*

We need to make sure the following FDs are violated, and violated such that all of their split FDs are also each violated (in other words, for $\{A\}\rightarrow\{B,C\}$, we want both $\{A\}\rightarrow\{B\}$ and $\{A\}\rightarrow\{C\}$ to be violated):

* $A_1\rightarrow\{A_2,A_3,A_4\}$
* $\{A_1,A_3\}\rightarrow\{A_2,A_4\}$
* $\{A_1,A_4\}\rightarrow\{A_2,A_3\}$
* $\{A_1,A_3,A_4\}\rightarrow A_2$
* $A_2\rightarrow\{A_1,A_3,A_4\}$
* $\{A_2,A_3\}\rightarrow\ A_1$
* $\{A_2,A_4\}\rightarrow\{A_1,A_3\}$
* $\{A_2,A_3,A_4\}\rightarrow A_1$
* $A_3\rightarrow\{A_1,A_2,A_4\}$
* $\{A_3,A_4\}\rightarrow\{A_1,A_2\}$
* $A_4\rightarrow\{A_1,A_2,A_3\}$

_One possible solution table is thus:_

In [10]:
%%sql
DROP TABLE IF EXISTS R; 
CREATE TABLE R (A int, B int, C int, D int);
INSERT INTO R VALUES (0, 0, 0, 0);
INSERT INTO R VALUES (0, 1, 1, 1);
INSERT INTO R VALUES (0, 2, 0, 2);
INSERT INTO R VALUES (0, 3, 3, 0);
INSERT INTO R VALUES (0, 4, 0, 0);
INSERT INTO R VALUES (5, 0, 5, 5);
INSERT INTO R VALUES (6, 0, 0, 6);
INSERT INTO R VALUES (7, 0, 7, 0);
INSERT INTO R VALUES (8, 0, 0, 0);
INSERT INTO R VALUES (9, 9, 0, 9);
INSERT INTO R VALUES (10, 10, 0, 0);
INSERT INTO R VALUES (11, 11, 11, 0);
SELECT * FROM R;

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


A,B,C,D
0,0,0,0
0,1,1,1
0,2,0,2
0,3,3,0
0,4,0,0
5,0,5,5
6,0,0,6
7,0,7,0
8,0,0,0
9,9,0,9


Problem 3
---------

**_[20 points total]_**

Consider a relation $R(X,Y,Z)$.  In each part of this problem you will be given a condition, and you need to create the following three instances of $R$ (as tables in SQL):

1. An instance $A$
2. An instance $B$ which differs from $A$ only in that it has one **_fewer_** row.
3. An instance $C$ which differs from $A$ only in that it has one **_additional_** row.

### Part (a)

**_[10 points]_**

Create $A$, $B$ and $C$ such that each individual attribute is a key for $A$, but none of the individual attributes is a key for $B$ or $C$.  If you believe that $B$ and/or $C$ cannot be created, provide them as an empty table.

In [1]:
%%sql
DROP TABLE IF EXISTS A;
CREATE TABLE A (X INT, Y INT, Z INT);
INSERT INTO A VALUES (0, 0, 0);
INSERT INTO A VALUES (1, 1, 1);
INSERT INTO A VALUES (2, 2, 2);

DROP TABLE IF EXISTS B;
CREATE TABLE B (X INT, Y INT, Z INT);

DROP TABLE IF EXISTS C;
CREATE TABLE C AS SELECT * FROM A;
INSERT INTO C VALUES (0, 1, 2);

ERROR: Cell magic `%%sql` not found.


### Part (b)

**_[10 points]_**

Create $A$, $B$ and $C$ such that the MVD $Z\twoheadrightarrow X$ holds in $A$, but not in $B$ or $C$.  If you believe that $B$ and/or $C$ cannot be created, provide them as an empty table.



In [3]:
%%sql
DROP TABLE IF EXISTS A;
CREATE TABLE A (X INT, Y INT, Z INT);
INSERT INTO A VALUES (1, 0, 0);
INSERT INTO A VALUES (0, 1, 0);
INSERT INTO A VALUES (1, 1, 0);
INSERT INTO A VALUES (0, 0, 0);

DROP TABLE IF EXISTS B;
CREATE TABLE B AS SELECT * FROM A;
DELETE FROM B WHERE A = 1 AND B = 0;

DROP TABLE IF EXISTS C;
CREATE TABLE C AS SELECT * FROM A;
INSERT INTO C VALUES (2, 0, 0);

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
(OperationalError) no such column: A u'DELETE FROM B WHERE A = 1 AND B = 0;' ()


Bonus Problem:
-------------

**_[10 points]_**

Prove the **transitivity rule for MVDs**

If $A\twoheadrightarrow B$ and $B\twoheadrightarrow C$ $\implies$ $A\twoheadrightarrow C \setminus B$

using only the basic definition of an MVD; and where $A,B,C$ are _sets of_ attributes such that $A\cup B\cup C\subseteq R$, where $R$ is the full set of attributes.

_Proof:_

1. Suppose that we have two tuples $t_1,t_2$ such that $t_1[A] = t_2[A]$

2. Because $A\twoheadrightarrow B$, we know by the definition of an MVD that $\exists t_3$ s.t.:
    1. $t_3[A] = t_1[A] = t_2[A]$
    2. $t_3[B] = t_1[B]$
    3. $t_3[R\setminus B]=t_2[R\setminus B]$

3. Since $B\twoheadrightarrow C$, there also $\exists t_4$ s.t.
    1. $t_4[B] = t_1[B] = t_3[B]$
    2. $t_4[C] = t_1[C]$
    3. $t_4[R\setminus C] = t_3[R\setminus C]$

4. To show that $A\twoheadrightarrow C \setminus B$, we would need to show that the following holds: Given $t_1[A] = t_3[A]$, there also $\exists t_4$ s.t.:
    1. $t_4[A] = t_1[A] = t_3[A]$
    2. $t_4[C\setminus B] = t_1[C\setminus B]$
    3. $t_4[R\setminus (C\setminus B)] = t_3[R\setminus (C\setminus B)]$

We now show that the 4.A-4.C hold:

4.A: We see that:

1. $t_4[R\setminus C] = t_3[R\setminus C]$ (3.C)
2. $\implies t_4[A\setminus C] = t_3[A\setminus C] = t_1[A\setminus C]$ (2.A)
3. We also know that $t_4[C] = t_1[C]$ (3.B)
4. $\implies t_4[(A\setminus C)\cup C] = t_1[(A\setminus C)\cup C]$
5. $\implies t_4[A]=t_1[A]$

4.B: $t_4[C] = t_1[C]$ (3.B) $\implies t_4[C\setminus B] = t_1[C\setminus B]$

4.C: We see that:

1. $t_4[R\setminus C] = t_3[R\setminus C]$ (3.C)
2. Also $t_4[B] = t_3[B]$ (3.A)
3. $\implies t_4[(R\setminus C)\cup B] = t_3[(R\setminus C)\cup B]$
4. $\implies t_4[R\setminus (C\setminus B)] = t_3[R\setminus (C\setminus B)]$
$\square$.